In [27]:
from bs4 import BeautifulSoup 
import numpy as np
import requests
import pandas as pd
import re
import math
import statistics as st
import time
from operator import itemgetter
import datetime
import os
import collections

In [55]:
url = input()
res = requests.get(url)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.content,features='html.parser')

https://race.netkeiba.com/race/shutuba.html?race_id=202205030411


In [58]:
horse_list = soup.find_all('tr', attrs={'class':'HorseList'})
race_name = soup.find('div', attrs={'class':'RaceName'}).text[:5]

In [59]:
horse_name = []
horse_url = []
umaban = []
wakuban = []
sex = []
age = []
kinryo = []
jockey = []
weight = []
weight_2 = []



for i in horse_list:
    try:
        wakuban.append(int(i.find_all('td')[0].find('span').text))
        umaban.append(int(i.find_all('td')[1].text))
    except ValueError:
        wakuban.append(i.find_all('td')[0].find('span').text)
        umaban.append(i.find_all('td')[1].text)
        
    horse_name.append(i.find('span', attrs={'class':'HorseName'}).text)
    horse_url.append(i.find('a').get('href'))
    for j, k in enumerate(i.find('td', attrs={'class':'Barei Txt_C'}).text):
        if j == 0:
            sex.append(k)
        else:
            age.append(int(k))
    kinryo.append(float(i.find_all('td', attrs={'class':'Txt_C'})[3].text))
    jockey.append(i.find('td', attrs={'class':'Jockey'}).text.strip())
    try:
        weight.append(int(i.find('td', attrs={'class':'Weight'}).text))
        weight_2.append(int(i.find('td', attrs={'class':'Weight'}).text))
    except ValueError:
        weight.append(i.find('td', attrs={'class':'Weight'}).text)
        weight_2.append(i.find('td', attrs={'class':'Weight'}).text)
    
    
    
    

In [60]:
papa, mama, mamapapa, pre_race,pre_race_info ,pre_race_result, time, tyakusa, pre_race_pop, corner_rank, pace, agari = [], [], [], [], [], [], [], [], [], [], [], []
for i in horse_url:
    res = requests.get(i)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content,features='html.parser')
    #血統の取得
    blood_table = soup.find('table', attrs={'class':'blood_table'})
    papa.append(blood_table.find_all('td', attrs={'class':'b_ml'})[0].text.strip())
    mama.append(blood_table.find_all('td', attrs={'class':'b_fml'})[1].text.strip())
    mamapapa.append(blood_table.find_all('td', attrs={'class':'b_ml'})[2].text.strip())
    #前走情報の取得
    race_results = soup.find('table', attrs={'class':'db_h_race_results nk_tb_common'}).find('tbody')
    if race_results.find_all('td')[11] != '除':
        pre_race.append(race_results.find_all('td')[4].text)
        pre_race_info.append(race_results.find_all('td')[14].text)
        pre_race_result.append(int(race_results.find_all('td')[11].text))
        time.append(race_results.find_all('td')[17].text)
        tyakusa.append(float(race_results.find_all('td')[18].text))
        pre_race_pop.append(int(race_results.find_all('td')[10].text))
        corner_rank.append(race_results.find_all('td')[20].text)
        pace.append(race_results.find_all('td')[21].text)
        agari.append(float(race_results.find_all('td')[22].text))
    else:
        pre_race.append(race_results.find_all('td')[32].text)
        pre_race_info.append(race_results.find_all('td')[42].text)
        pre_race_result.append(int(race_results.find_all('td')[39].text))
        time.append(race_results.find_all('td')[45].text)
        tyakusa.append(float(race_results.find_all('td')[46].text))
        pre_race_pop.append(int(race_results.find_all('td')[38].text))
        corner_rank.append(race_results.find_all('td')[48].text)
        pace.append(race_results.find_all('td')[49].text)
        agari.append(float(race_results.find_all('td')[50].text))




In [61]:
df = pd.DataFrame(
    {
        '馬番':umaban,
        '枠番':wakuban,
        '馬名':horse_name,
        '性':sex,
        '齢':age,
        '斤量':kinryo,
        '騎手':jockey,
        '馬体重':weight,
        '馬体重増減':weight_2,
        '父':papa,
        '母':mama,
        '母父':mamapapa,
        '前走':pre_race,
        '前走距離':pre_race_info,
        '前走着順':pre_race_result,
        '前走人気':pre_race_pop,
        '前走タイム':time,
        '前走着差':tyakusa,
        '前走通過順位':corner_rank,
        '前走ペース':pace,
        '前走上がり':agari
    }
)

In [62]:
dir_path = "C:/Users/sashi/OneDrive/デスクトップ/競馬データ/"+race_name+"/出馬表.xlsx"
with pd.ExcelWriter(dir_path) as writer:
    df.to_excel(writer,sheet_name='出馬表',encoding='shift_jis',index=False)